#                    Машинное обучение для текстов

####  <font color='red'>Описание :</font>
Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

####  <font color='red'>Что надо сделать:</font>
Обучить модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

####  <font color='red'>Цель:</font> 
Построить модель со значением метрики качества F1 не меньше 0.75. 

# 1. Загрузка и подготовка данных

## 1.1. Загрузка и установка необходимых библиотек

In [1]:
!pip install spacy

In [2]:
!pip install --upgrade spacy

  Using cached spacy-3.6.1-cp310-cp310-win_amd64.whl (12.0 MB)
     ---------------------------------------- 1.5/1.5 MB 989.5 kB/s eta 0:00:00
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found existing installation: spacy 3.0.9
    Uninstalling spacy-3.0.9:
      Successfully uninstalled spacy-3.0.9


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.0.0 requires spacy<3.1.0,>=3.0.0, but you have spacy 3.6.1 which is incompatible.


In [3]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0-py3-none-any.whl --user

     ---------------------------------------- 13.7/13.7 MB 1.2 MB/s eta 0:00:00
  Using cached spacy-3.0.9-cp310-cp310-win_amd64.whl (11.1 MB)
  Using cached thinc-8.0.17-cp310-cp310-win_amd64.whl (1.0 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## 1.2. Импорт необходимых библиотек

In [4]:
#основное
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
#предобработка
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [6]:
#токенизация
import spacy
import nltk
from nltk.stem import SnowballStemmer

In [7]:
#модели
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier, Pool, cv


In [8]:
#обучение
from sklearn.model_selection import train_test_split, GridSearchCV
from tune_sklearn import TuneGridSearchCV

In [74]:
#метрики
from sklearn.metrics import f1_score, accuracy_score

In [10]:
#разное
plt.style.use('dark_background')
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

## 1.3. Загрузка данных

In [11]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [12]:
df_raw = df.copy()

In [13]:
df_raw.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [14]:
df_raw

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


Первое, что мы видим - твитты на английском языке. Второе - они содержат много ненужных слов. Третье - датасет довольно большой. В связи с этим надо будет решить каким образом проводить его токенизацию, что это не заняло слишком много времени и ресурсов компьютера. Но сначала выполним предобработку твиттов. 

In [15]:
df_raw['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Один класс преобладает над другим. Необходимо будет это учесть при обучении моеделей.

## 1.4. Очитска текста приведение слов к стандартному виду

Приведем столбец к типу list

In [16]:
df_raw['text'] = list(df_raw['text'])

Напишим функцию, удаляющую все кроме букв, цифр и пробелов

In [20]:
def df_preprocess(text):
    a = re.sub(r'[+\W]', '', text) # удаляем все символы кроме букв, цифр, знаков подчеркивания и пробелов
    b = re.findall(r'\w+', text) # находим все слова в тексте
    return ' '.join([x for x in b if x.isalnum()]) # склеиваем найденные слова с пробелом между ними

In [18]:
df_raw['text'] = df_raw['text'].apply(df_preprocess)

In [19]:
df_raw

,Unnamed: 0,text,toxic
0,0,Explanation Why the edits made under my userna...,0
1,1,D aww He matches this background colour I m se...,0
2,2,Hey man I m really not trying to edit war It s...,0
3,3,More I can t make any real suggestions on impr...,0
4,4,You sir are my hero Any chance you remember wh...,0
...,...,...,...
159287,159446,And for the second time of asking when your vi...,0
159288,159447,You should be ashamed of yourself That is a ho...,0
159289,159448,Spitzer Umm theres no actual article for prost...,0
159290,159449,And it looks like it was actually you who put ...,0


Исчезли знаки препинания.

In [ ]:
df_lem = df_raw.copy()

## 1.5. Токенизация текста

Выполним токенизацию текста с помощью лемм. Посмотрим сколько времени это может занять. Поскольку твитты на английском языке для лемматизации будем использовать библиотеку spacy.

In [27]:
nlp = spacy.load("en_core_web_sm")

In [23]:
df_lem = df_raw.copy()

Проверим как работает функция.

In [24]:
df_lem

,Unnamed: 0,text,toxic
0,0,Explanation Why the edits made under my userna...,0
1,1,D aww He matches this background colour I m se...,0
2,2,Hey man I m really not trying to edit war It s...,0
3,3,More I can t make any real suggestions on impr...,0
4,4,You sir are my hero Any chance you remember wh...,0
...,...,...,...
159287,159446,And for the second time of asking when your vi...,0
159288,159447,You should be ashamed of yourself That is a ho...,0
159289,159448,Spitzer Umm theres no actual article for prost...,0
159290,159449,And it looks like it was actually you who put ...,0


In [25]:
new_corpus = []

for doc in tqdm(nlp.pipe(df_lem['text'], batch_size=64, n_process=-1, disable=["parser", "ner"]), total=len(df_lem['text'])):
    word_list = [tok.lemma_ for tok in doc]
    new_corpus.append(' '.join(word_list))
    
df_lem['lemm_spacy_new'] = new_corpus  

100%|█████████████████████████████████████████████████████████████████████████| 159292/159292 [06:33<00:00, 404.32it/s]


Значительно быстрее. Далее посмотрим как это повлияет на определение тональности твиттов.

In [26]:
df_lem

,Unnamed: 0,text,toxic,lemm_spacy_new
0,0,Explanation Why the edits made under my userna...,0,Explanation why the edit make under my usernam...
1,1,D aww He matches this background colour I m se...,0,d aww he match this background colour I m seem...
2,2,Hey man I m really not trying to edit war It s...,0,hey man I m really not try to edit war it s ju...
3,3,More I can t make any real suggestions on impr...,0,More I can t make any real suggestion on impro...
4,4,You sir are my hero Any chance you remember wh...,0,you sir be my hero any chance you remember wha...
...,...,...,...,...
159287,159446,And for the second time of asking when your vi...,0,and for the second time of ask when your view ...
159288,159447,You should be ashamed of yourself That is a ho...,0,you should be ashamed of yourself that be a ho...
159289,159448,Spitzer Umm theres no actual article for prost...,0,Spitzer Umm there s no actual article for pros...
159290,159449,And it looks like it was actually you who put ...,0,and it look like it be actually you who put on...


In [27]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aleksey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 1.6. Векторизация текста

Применим TfidfVectorizer с английским списком стоп слов.

In [28]:
vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'))

Разделим данные на обучающую и тестовую выборки.

In [29]:
train, test = train_test_split(df_lem, test_size=0.2)

In [30]:
X_train = train['lemm_spacy_new']
y_train = train['toxic']
X_test = test['lemm_spacy_new']
y_test = test['toxic']

Обучим модель.

In [31]:
vectorizer.fit(X_train)

TfidfVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

Применим обученную модель для трансформации признаков обучающей и тестовой выборок.

In [32]:
X_train_vec = vectorizer.transform(X_train)

In [33]:
X_test_vec = vectorizer.transform(X_test)

# 2. Определение лучшей модели

### 2.1. Логистическая регрессия 

Применим кросс-валидацию и GridSearch

In [34]:
param = { 'C': range(1, 11, 2), 'class_weight': [None, 'balanced'] }

model_lr = LogisticRegression()

# инициализируем GridSearchCV
cv_lr = GridSearchCV(estimator = model_lr, 
                           param_grid = param, 
                           cv = 3,
                           n_jobs = -1, 
                           verbose = 0, 
                           scoring = 'f1',
                          )
cv_lr.fit(X_train_vec, y_train)  

GridSearchCV(cv=3, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': range(1, 11, 2),
                         'class_weight': [None, 'balanced']},
             scoring='f1')

In [35]:
# Лучшие параметры и лучшая оценка модели
best_params_lr = cv_lr.best_params_
print(f'Лучшие параметры модели:{best_params_lr}')

best_score_lr = cv_lr.best_score_
print(f'Лучшая метрика f1 на кросс-валидации:{best_score_lr}')

best_model_lg = cv_lr.best_estimator_

Лучшие параметры модели:{'C': 9, 'class_weight': 'balanced'}
Лучшая метрика f1 на кросс-валидации:0.7673059430220284


## 2.2. CatBoostClassifier

Применим кросс-валидацию

In [36]:
# Создаем Pool с данными
train_data = Pool(data=X_train_vec, label=y_train)

# Указываем параметры модели
params = {
    'eval_metric': 'F1',
    'loss_function': 'Logloss',
    'learning_rate': 0.05,
    'random_seed': 2007,
    'verbose': 100
}

# Проводим кросс-валидацию
cv_data = cv(params=params,
             pool=train_data,
             fold_count=3,
             shuffle=True,
             partition_random_seed=0,
             stratified=False,
             verbose=False,
             early_stopping_rounds=200)

# Создаем модель CatBoostClassifier
model_cb = CatBoostClassifier(**params)

# Обучаем модель CatBoostClassifier
model_cb.fit(X_train_vec, y_train)

Training on fold [0/3]

bestTest = 0.7355268638
bestIteration = 997

Training on fold [1/3]

bestTest = 0.7373639994
bestIteration = 994

Training on fold [2/3]

bestTest = 0.7413984462
bestIteration = 999

0:	learn: 0.4517345	total: 925ms	remaining: 15m 23s
100:	learn: 0.6163509	total: 1m 21s	remaining: 12m 5s
200:	learn: 0.6654965	total: 2m 39s	remaining: 10m 33s
300:	learn: 0.6965276	total: 3m 56s	remaining: 9m 9s
400:	learn: 0.7186754	total: 5m 14s	remaining: 7m 49s
500:	learn: 0.7328367	total: 6m 32s	remaining: 6m 30s
600:	learn: 0.7436928	total: 7m 49s	remaining: 5m 11s
700:	learn: 0.7528965	total: 9m 7s	remaining: 3m 53s
800:	learn: 0.7626091	total: 10m 25s	remaining: 2m 35s
900:	learn: 0.7698626	total: 11m 42s	remaining: 1m 17s
999:	learn: 0.7746248	total: 12m 59s	remaining: 0us


Найдем best_score для CatBoostClassifier

In [37]:
best_score_cb = cv_data['test-F1-mean'].max()

In [39]:
round(best_score_cb,3)

0.738

Как видно на кросс-валидации лучшее значение метрики F1 у логистической регрессии. Ее и возьмем для проверки на тестовых данных.

In [41]:
# Предсказание результатов для тестовой выборки
predictions = best_model_lg.predict(X_test_vec)

In [44]:
print('Значение F1 для модели логистической регрессии составило', \
      round(f1_score(y_test, predictions),3))

Значение F1 для модели CatBoostClassifier после лемматизации составило 0.763


## 3. KERAS

Для проверки корректности работы модели и экономии времени возьмем часть данных - 10% от общего количества твиттов. Хотя конечно для нейросетей чем больше обучающих данных, тем лучше.

In [17]:
df_kr = df_raw.sample(frac=0.1)

In [18]:
df_kr['toxic'].value_counts()

0    14307
1     1622
Name: toxic, dtype: int64

Соотношение целевых переменных сохранилось.

In [19]:
df_kr['text'] = list(df_kr['text'])

In [21]:
df_kr['text'] = df_kr['text'].apply(df_preprocess)

In [28]:
new_corpus_kr = []

for doc in tqdm(nlp.pipe(df_kr['text'], batch_size=64, n_process=-1, disable=["parser", "ner"]), total=len(df_kr['text'])):
    word_list = [tok.lemma_ for tok in doc]
    new_corpus_kr.append(' '.join(word_list))
    
df_kr['lemm_spacy_new'] = new_corpus_kr  

100%|███████████████████████████████████████████████████████████████████████████| 15929/15929 [01:54<00:00, 138.58it/s]


In [29]:
!pip install keras

In [30]:
!pip install tensorflow

In [31]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.utils import pad_sequences

In [78]:
train, test = train_test_split(df_kr, test_size=0.2)

In [118]:
X_train = train['lemm_spacy_new']
y_train = train['toxic']
X_test = test['lemm_spacy_new']
y_test = test['toxic']

In [119]:
# Создание токенизатора
tokenizer = Tokenizer()

In [120]:
# Обучение токенизатора
tokenizer.fit_on_texts(X_train)

In [121]:
tokenizer.fit_on_texts(X_test)

In [122]:
# Преобразование текстов в последовательности чисел
sequences = tokenizer.texts_to_sequences(X_train)

In [123]:
# Заполнение последовательностей до одинаковой длины
max_length = max([len(seq) for seq in sequences])
data = pad_sequences(sequences, maxlen=max_length)

In [124]:
# Создание модели нейронной сети
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=max_length))
model.add(LSTM(50))
model.add(Dense(1, activation='sigmoid'))

In [125]:
# настройка модели 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [126]:
# Обучение модели
model.fit(data, y_train, epochs=1, batch_size=2)

6372/6372 [==============================] - 1346s 211ms/step - loss: 0.2009 - accuracy: 0.9340


In [127]:
new_sequences = tokenizer.texts_to_sequences(X_test)

In [128]:
new_data = pad_sequences(new_sequences, maxlen=max_length)

In [129]:
# Предсказание результатов для тестовой выборки
predictions = model.predict(new_data)

100/100 [==============================] - 9s 89ms/step


In [130]:
predictions_kr = []
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        predictions_kr.append(1)
    elif predictions[i] < 0.5:
        predictions_kr.append(0)

In [134]:
print('Значение F1 для модели Keras составило', \
      round(f1_score(y_test, predictions_kr),3))

Значение F1 для модели Keras составило 0.743


## 4. Вывод

Лучший результат на кросс-валидации показала логистическая регрессия со значением F1 = 0.767. На втором месте оказался CatBoost со значением F1 = 0.738. На третьем - случайный лес с F1 = 0.65. Также следует отметить, что для ускорения процесса лусше использовать стемминг, а не лемматизацию. 
Для использования на тестовой выборке была выбрана модель логистической регрессии, которая показал значения F1 = 0.763.